In [1]:
import pandas as pd
import os
import mrcfile
import csv
import numpy as np

In [4]:
def conv_phase_to_mrc1(phasefile,reffile,keyidx,outmapfile):
    if os.path.isfile(outmapfile):
        return True
    INPUT=[]
    with open(phasefile) as f:
        reader = csv.reader(f)
        for row in reader:
            #xyz:[0,3]
            INPUT.append([int(float(row[0])),int(float(row[1])),int(float(row[2])),float(row[keyidx+3])])
    with mrcfile.open(reffile) as mrc:
        nx,ny,nz,nxs,nys,nzs,mx,my,mz,cella = mrc.header.nx,mrc.header.ny,mrc.header.nz,mrc.header.nxstart,mrc.header.nystart,mrc.header.nzstart,mrc.header.mx,mrc.header.my,mrc.header.mz, mrc.header.cella
        orig=mrc.header.origin
        data=mrc.data
        data_new= data #init
        #update data by INPUT
        #mrc_new = mrcfile.new(outmapfile,data=data_new, overwrite=True)
        
        mrc_new = mrcfile.new(outmapfile,overwrite=True)
        
        mrc_new.set_data(np.zeros((nz, ny, nx), dtype=np.float32))
        mrc_new.header.nxstart=nxs
        mrc_new.header.nystart=nys
        mrc_new.header.nzstart=nzs
        mrc_new.header.cella['x'] = cella['x'] / 1.0700002
        mrc_new.header.cella['y'] = cella['y'] / 1.0700002
        mrc_new.header.cella['z'] = cella['z'] / 1.0700002
        
        for x,y,z,prob in INPUT:
            mrc_new.data[z,y,x]=prob
        vsize=mrc_new.voxel_size
        vsize.flags.writeable = True
        mrc_new.voxel_size=vsize         
         #             mrc_new.header.xorg=float(xx[0]) - float(mx)/2.0
        
        mrc_new.update_header_stats()
        
        print("original", mrc.voxel_size)
        mrc.print_header()

        print()
        
        print("new", mrc_new.voxel_size)
        mrc_new.print_header()

        print()
        
        mrc_new.close()    

In [5]:
structures = [1, 2, 3, 4]
csv_path = "Input_emd_8097_chainB_centered.csv"
mrc_path = "centered_8097_chainB_resampled.mrc"
map_name = "emd_8097_chainB"

for i in structures:
    if i == 1:
        name = "_helix.mrc"
    elif i == 2:
        name = "_coil.mrc"
    elif i == 3:
        name = "_strand.mrc"
    else:
        name = "_DNA.mrc"
      
    conv_phase_to_mrc1(csv_path, mrc_path, i, map_name + name)

original (1., 1., 1.)
nx              : 144
ny              : 117
nz              : 200
mode            : 2
nxstart         : 0
nystart         : 0
nzstart         : 0
mx              : 144
my              : 117
mz              : 200
cella           : (144., 117., 200.)
cellb           : (90., 90., 90.)
mapc            : 1
mapr            : 2
maps            : 3
dmin            : 0.0
dmax            : 0.5373286604881287
dmean           : 0.007735585793852806
ispg            : 1
nsymbt          : 0
extra1          : b'\x00\x00\x00\x00\x00\x00\x00\x00'
exttyp          : b''
nversion        : 20140
extra2          : b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'
origin          : (0., 0., 0.)
map     